In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse.csgraph import dijkstra
from scipy.sparse import csr_matrix
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

np.random.seed(42)
tf.random.set_seed(42)

In [2]:
# Set up the log directory
import tensorboard
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Create TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

### Dataset operations

In [3]:
data = pd.read_csv('C:/Code_Win/ML_DL_RL/Dataset/3sat_LSTM_RTT/Dataset_3sat.csv')
data['Time_datetime'] = pd.to_datetime(data['Time'])
ref_time = data['Time_datetime'].iloc[0]
data['OBT'] = (data['Time_datetime'] - ref_time).dt.total_seconds()
data = data.drop(['Time_datetime'], axis=1)
columns_sorted = sorted(data.columns.difference(['OBT', 'Time']))
new_order = ['Time', 'OBT'] + columns_sorted
data = data[new_order]
data.to_csv('C:/Code_Win/ML_DL_RL/Dataset/5sat_LSTM_RTT/Dataset_2sat_OBT.csv', index=False)
data = pd.read_csv('C:/Code_Win/ML_DL_RL/Dataset/5sat_LSTM_RTT/Dataset_2sat_OBT.csv')
data.head(20)

C:\Users\gmath\AppData\Local\Temp\ipykernel_7252\1648026555.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Time_datetime'] = pd.to_datetime(data['Time'])


,Time,OBT,RTT_Bangalore_Bangalore,RTT_Bangalore_CARTOSAT2C,RTT_Bangalore_Resourcesat2,RTT_CARTOSAT2C_Bangalore,RTT_CARTOSAT2C_CARTOSAT2C,RTT_CARTOSAT2C_Resourcesat2,RTT_Resourcesat2_Bangalore,RTT_Resourcesat2_CARTOSAT2C,RTT_Resourcesat2_Resourcesat2
0,00:00:01,0.0,NaN,NaN,NaN,NaN,NaN,8.2481,NaN,8.2485,NaN
1,00:00:02,1.0,NaN,NaN,NaN,NaN,NaN,8.2497,NaN,8.2501,NaN
2,00:00:03,2.0,NaN,NaN,NaN,NaN,NaN,8.2513,NaN,8.2517,NaN
3,00:00:04,3.0,NaN,NaN,NaN,NaN,NaN,8.2530,NaN,8.2534,NaN
4,00:00:05,4.0,NaN,NaN,NaN,NaN,NaN,8.2546,NaN,8.2550,NaN
5,00:00:06,5.0,NaN,NaN,NaN,NaN,NaN,8.2562,NaN,8.2566,NaN
6,00:00:07,6.0,NaN,NaN,NaN,NaN,NaN,8.2578,NaN,8.2582,NaN
7,00:00:08,7.0,NaN,NaN,NaN,NaN,NaN,8.2595,NaN,8.2599,NaN
8,00:00:09,8.0,NaN,NaN,NaN,NaN,NaN,8.2611,NaN,8.2615,NaN
9,00:00:10,9.0,NaN,NaN,NaN,NaN,NaN,8.2627,NaN,8.2631,NaN


In [4]:
numerical_data = data.iloc[:, 2:].select_dtypes(include=[float])
data_max = numerical_data.stack().max()
data_min = numerical_data.stack().min()
data_max, data_min

(19.8848, 2.4829)

In [5]:
data.iloc[:, 1:] = data.iloc[:, 1:].round(3)
# data.replace(float('inf'), float(20), inplace=True)
data.fillna(np.inf, inplace=True)
data.to_csv('C:/Code_Win/ML_DL_RL/Dataset/5sat_LSTM_RTT/Dataset_2sat_inf.csv', index=False)
data.head(20)

,Time,OBT,RTT_Bangalore_Bangalore,RTT_Bangalore_CARTOSAT2C,RTT_Bangalore_Resourcesat2,RTT_CARTOSAT2C_Bangalore,RTT_CARTOSAT2C_CARTOSAT2C,RTT_CARTOSAT2C_Resourcesat2,RTT_Resourcesat2_Bangalore,RTT_Resourcesat2_CARTOSAT2C,RTT_Resourcesat2_Resourcesat2
0,00:00:01,0.0,inf,inf,inf,inf,inf,8.248,inf,8.248,inf
1,00:00:02,1.0,inf,inf,inf,inf,inf,8.250,inf,8.250,inf
2,00:00:03,2.0,inf,inf,inf,inf,inf,8.251,inf,8.252,inf
3,00:00:04,3.0,inf,inf,inf,inf,inf,8.253,inf,8.253,inf
4,00:00:05,4.0,inf,inf,inf,inf,inf,8.255,inf,8.255,inf
5,00:00:06,5.0,inf,inf,inf,inf,inf,8.256,inf,8.257,inf
6,00:00:07,6.0,inf,inf,inf,inf,inf,8.258,inf,8.258,inf
7,00:00:08,7.0,inf,inf,inf,inf,inf,8.260,inf,8.260,inf
8,00:00:09,8.0,inf,inf,inf,inf,inf,8.261,inf,8.262,inf
9,00:00:10,9.0,inf,inf,inf,inf,inf,8.263,inf,8.263,inf


### Predecessor Matrx Generation

In [6]:
def get_predecessors(rtt_values):
    graph = csr_matrix(rtt_values.astype(np.float64))  # Ensure data is float64
    _, predecessors = dijkstra(rtt_values, return_predecessors=True, directed=False)
    return predecessors

predecessors_list = []

# Iterating over rows, converting each row to numpy array, and ensuring float64 dtype
for _, row in data.iterrows():
    rtt_values = row[2:].values.reshape(3, 3).astype(np.float64)
    predecessors_list.append(get_predecessors(rtt_values).flatten())

Y = np.array(predecessors_list)

In [7]:
Y

array([[-9999, -9999, -9999, ..., -9999,     2, -9999],
       [-9999, -9999, -9999, ..., -9999,     2, -9999],
       [-9999, -9999, -9999, ..., -9999,     2, -9999],
       ...,
       [-9999, -9999, -9999, ..., -9999,     2, -9999],
       [-9999, -9999, -9999, ..., -9999,     2, -9999],
       [-9999, -9999, -9999, ..., -9999,     2, -9999]])

#### Predecessor Verification

In [8]:
print(Y.shape)
print(Y[11780], '\n', Y[11780].reshape(3, 3), '\n') 
temp = data.iloc[11780, 2:].values.reshape(3, 3).astype(np.float64)
print(data.iloc[11780, 2:], '\n', get_predecessors(temp))

(27330, 9)
[-9999     0 -9999     1 -9999 -9999 -9999 -9999 -9999] 
 [[-9999     0 -9999]
 [    1 -9999 -9999]
 [-9999 -9999 -9999]] 

RTT_Bangalore_Bangalore            inf
RTT_Bangalore_CARTOSAT2C         7.501
RTT_Bangalore_Resourcesat2         inf
RTT_CARTOSAT2C_Bangalore         7.501
RTT_CARTOSAT2C_CARTOSAT2C          inf
RTT_CARTOSAT2C_Resourcesat2        inf
RTT_Resourcesat2_Bangalore         inf
RTT_Resourcesat2_CARTOSAT2C        inf
RTT_Resourcesat2_Resourcesat2      inf
Name: 11780, dtype: object 
 [[-9999     0 -9999]
 [    1 -9999 -9999]
 [-9999 -9999 -9999]]


In [9]:
print(Y.shape)
print(Y[0], '\n', Y[0].reshape(3, 3), '\n') 
temp = data.iloc[0][2:].to_numpy().reshape(3, 3).astype(np.float64)
print(get_predecessors(temp))

(27330, 9)
[-9999 -9999 -9999 -9999 -9999     1 -9999     2 -9999] 
 [[-9999 -9999 -9999]
 [-9999 -9999     1]
 [-9999     2 -9999]] 

[[-9999 -9999 -9999]
 [-9999 -9999     1]
 [-9999     2 -9999]]


### Custom Scaler

In [10]:
class MyMinMaxScaler:
    def __init__(self, data_min=None, data_max=None):
        self.min_ = data_min
        self.max_ = data_max

    def fit(self, data):
        # If self.min_ or self.max_ hasn't been provided externally, compute it
        if self.min_ is None:
            self.min_ = data.min()
        if self.max_ is None:
            self.max_ = data.max()

    def transform(self, data):
        if self.min_ is None or self.max_ is None:
            raise ValueError("Scaler has not been fitted yet or min/max values were not provided.")
        return (data - self.min_) / (self.max_ - self.min_)

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

    def inverse_transform(self, data):
        if self.min_ is None or self.max_ is None:
            raise ValueError("Scaler has not been fitted yet or min/max values were not provided.")
        return data * (self.max_ - self.min_) + self.min_

In [11]:
MAX_REPLACE = (data_max*10).round(3)
data.replace(np.inf, MAX_REPLACE, inplace=True)
data.head(20)
X = data.iloc[:, 2:].values.astype(np.float64)
X   

array([[198.848, 198.848, 198.848, ..., 198.848,   8.248, 198.848],
       [198.848, 198.848, 198.848, ..., 198.848,   8.25 , 198.848],
       [198.848, 198.848, 198.848, ..., 198.848,   8.252, 198.848],
       ...,
       [198.848, 198.848, 198.848, ..., 198.848,   5.799, 198.848],
       [198.848, 198.848, 198.848, ..., 198.848,   5.801, 198.848],
       [198.848, 198.848, 198.848, ..., 198.848,   5.803, 198.848]])

In [12]:
scaler = MyMinMaxScaler(data_min=data_min, data_max=MAX_REPLACE)
scaled_data = scaler.transform(X)
scaled_data.shape

(27330, 9)

In [13]:
scaler_OBT = MyMinMaxScaler(data_min=data['OBT'].min(), data_max=data['OBT'].max())
scaled_OBT = scaler_OBT.transform(data['OBT'])
scaled_OBT.shape

(27330,)

#### Verifying Scaled Values

In [14]:
print(X[11780], '\n',  X[0])

[198.848   7.501 198.848   7.501 198.848 198.848 198.848 198.848 198.848] 
 [198.848 198.848 198.848 198.848 198.848   8.248 198.848   8.248 198.848]


In [15]:
print(data.iloc[23], '\n', scaled_data[23])

Time                             00:00:24
OBT                                  23.0
RTT_Bangalore_Bangalore           198.848
RTT_Bangalore_CARTOSAT2C          198.848
RTT_Bangalore_Resourcesat2        198.848
RTT_CARTOSAT2C_Bangalore          198.848
RTT_CARTOSAT2C_CARTOSAT2C         198.848
RTT_CARTOSAT2C_Resourcesat2         8.286
RTT_Resourcesat2_Bangalore        198.848
RTT_Resourcesat2_CARTOSAT2C         8.286
RTT_Resourcesat2_Resourcesat2     198.848
Name: 23, dtype: object 
 [1.        1.        1.        1.        1.        0.0295526 1.
 0.0295526 1.       ]


### Sliding Window

In [16]:
# def create_dataset(X, y, window_size=1):
#     dataX, dataY = [], []
#     for i in range(len(X) - window_size - 1):  # -1 to get the future value for Y
#         # Past observations based on window size
#         a = X[i:(i+window_size), :]
#         # Current timestamp, source, and destination nodes
#         current_values = X[i+window_size, :]
#         full_input = np.vstack((a, current_values))
#         dataX.append(full_input)
#         future_value = y[i+window_size]
#         dataY.append(future_value)
#     return np.array(dataX), np.array(dataY)

In [17]:
def create_dataset(X, y, obt, window_size=1):
    dataX, dataY, dataOBT = [], [], []
    for i in range(len(X) - window_size - 1):
        # Past observations based on window size
        a = X[i:(i+window_size), :]
        # Current timestamp, source, and destination nodes
        current_values = X[i+window_size, :]
        full_input = np.vstack((a, current_values))
        dataX.append(full_input)
        
        # Corresponding OBT values for the sequence
        obt_sequence = obt[i:(i+window_size+1)]  # +1 to include the current value
        dataOBT.append(obt_sequence)
        
        future_value = y[i+window_size]
        dataY.append(future_value)
        
    return np.array(dataX), np.array(dataY), np.array(dataOBT)

In [27]:
window_size = 10  
X_window, Y_window, OBT_window = create_dataset(scaled_data, Y, scaled_OBT, window_size)   
X_window.shape, Y_window.shape, OBT_window.shape

((27319, 11, 9), (27319, 9), (27319, 11))

In [19]:
print(type(X_window))

<class 'numpy.ndarray'>


### Model Creation

In [20]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dropout, Dense
# from tensorflow.keras.optimizers import Adam

# model = Sequential()
# model.add(LSTM(128, return_sequences=True, input_shape=(X_window.shape[1], X_window.shape[2])))
# model.add(Dropout(0.3))
# model.add(LSTM(64, return_sequences=True))
# model.add(Dropout(0.3))
# model.add(LSTM(64))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(Y_window.shape[1]))
# model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# model.summary()

In [21]:
# from tensorflow.keras.layers import Input, concatenate
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import LSTM, Dropout, Dense
# from tensorflow.keras.optimizers import Adam

# # Define the two sets of inputs
# main_input = Input(shape=(X_window.shape[1], X_window.shape[2]), name='main_input')
# obt_input = Input(shape=(OBT_window.shape[1],), name='obt_input')

# # First path: LSTM layers for main input
# x = LSTM(128, return_sequences=True)(main_input)
# x = Dropout(0.3)(x)
# x = LSTM(64, return_sequences=True)(x)
# x = Dropout(0.3)(x)
# x = LSTM(64)(x)

# main_output = Dense(32, activation='relu')(x)
# y = Dense(32, activation='relu')(obt_input) # Second path: Dense layer for OBT input
# combined = concatenate([main_output, y]) # Combine the output of the two branches

# z = Dense(64, activation='relu')(combined)
# z = Dropout(0.3)(z)
# z = Dense(32, activation='relu')(z)
# # Final output
# final_output = Dense(Y_window.shape[1])(z)

# model = Model(inputs=[main_input, obt_input], outputs=final_output)
# model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# model.summary()

In [32]:
# from tensorflow.keras.layers import Input, concatenate
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import LSTM, Dropout, Dense
# from tensorflow.keras.optimizers import Adam

# # Define the two sets of inputs
# main_input = Input(shape=(X_window.shape[1], X_window.shape[2]), name='main_input')
# obt_input = Input(shape=(OBT_window.shape[1],), name='obt_input')

# # First path: LSTM layers for main input
# x = LSTM(128, return_sequences=True)(main_input)
# x = Dropout(0.5)(x)
# x = LSTM(64, return_sequences=True)(x)
# x = Dropout(0.5)(x)
# x = LSTM(64)(x)

# main_output = Dense(32, activation='relu')(x)
# y = Dense(32, activation='relu')(obt_input) # Second path: Dense layer for OBT input
# combined = concatenate([main_output, y]) # Combine the output of the two branches

# final_output = Dense(Y_window.shape[1])(combined)

# model = Model(inputs=[main_input, obt_input], outputs=final_output)
# model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# model.summary()

In [34]:
# from tensorflow.keras.layers import Input, concatenate, BatchNormalization
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import LSTM, Dropout, Dense
# from tensorflow.keras.optimizers import Adam

# # Define the two sets of inputs
# main_input = Input(shape=(X_window.shape[1], X_window.shape[2]), name='main_input')
# obt_input = Input(shape=(OBT_window.shape[1],), name='obt_input')

# # First path: LSTM layers for main input
# x = LSTM(128, return_sequences=True)(main_input)
# x = BatchNormalization()(x) # Added BatchNormalization
# x = Dropout(0.5)(x)
# x = LSTM(64, return_sequences=True)(x)
# x = BatchNormalization()(x) # Added BatchNormalization
# x = Dropout(0.5)(x)
# x = LSTM(64)(x)
# x = BatchNormalization()(x) # Added BatchNormalization

# main_output = Dense(32, activation='relu')(x)
# y = Dense(32, activation='relu')(obt_input) 
# y = BatchNormalization()(y) # Added BatchNormalization for the OBT path

# combined = concatenate([main_output, y]) # Combine the output of the two branches

# final_output = Dense(Y_window.shape[1])(combined)

# model = Model(inputs=[main_input, obt_input], outputs=final_output)
# model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 main_input (InputLayer)     [(None, 11, 9)]              0         []                            
                                                                                                  
 lstm_12 (LSTM)              (None, 11, 128)              70656     ['main_input[0][0]']          
                                                                                                  
 batch_normalization (Batch  (None, 11, 128)              512       ['lstm_12[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 dropout_8 (Dropout)         (None, 11, 128)              0         ['batch_normalization[0]

In [40]:
from tensorflow.keras.layers import Input, concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam

# Define the two sets of inputs
main_input = Input(shape=(X_window.shape[1], X_window.shape[2]), name='main_input')
obt_input = Input(shape=(OBT_window.shape[1],), name='obt_input')

# First path: LSTM layers for main input
x = LSTM(128, return_sequences=True)(main_input)
x = Dropout(0.5)(x)
x = LSTM(64, return_sequences=True)(x)
x = Dropout(0.5)(x)
x = LSTM(64)(x)
x = BatchNormalization()(x) # Added just one BatchNormalization after the final LSTM layer

main_output = Dense(32, activation='relu')(x)
y = Dense(32, activation='relu')(obt_input) 
y = BatchNormalization()(y) # Added BatchNormalization for the OBT path

combined = concatenate([main_output, y]) # Combine the output of the two branches

final_output = Dense(Y_window.shape[1])(combined)

model = Model(inputs=[main_input, obt_input], outputs=final_output)
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 main_input (InputLayer)     [(None, 11, 9)]              0         []                            
                                                                                                  
 lstm_15 (LSTM)              (None, 11, 128)              70656     ['main_input[0][0]']          
                                                                                                  
 dropout_10 (Dropout)        (None, 11, 128)              0         ['lstm_15[0][0]']             
                                                                                                  
 lstm_16 (LSTM)              (None, 11, 64)               49408     ['dropout_10[0][0]']          
                                                                                            

### Model Training, Saving, Loading and Predection

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_window, Y_window, test_size=0.2, shuffle=False)
OBT_train, OBT_val = train_test_split(OBT_window, test_size=0.2, shuffle=False)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, OBT_train.shape, OBT_val.shape  

((21855, 11, 9), (5464, 11, 9), (21855, 9), (5464, 9), (21855, 11), (5464, 11))

In [ ]:
X_train[0], y_train[0], OBT_train[0]

In [ ]:
X_train[100], y_train[100], OBT_train[100]

In [42]:
from sklearn.metrics import mean_squared_error
from math import sqrt

predictions = model.predict([X_val, OBT_val])
mse = sqrt(mean_squared_error(y_val, predictions))

predictions.shape, mse

171/171 [==============================] - 4s 6ms/step


((5464, 9), 8817.98353170075)

In [43]:
model.fit([X_train, OBT_train], y_train, 
          epochs=100, 
          batch_size=64, 
          validation_data=([X_val, OBT_val], y_val), 
          verbose=2, 
          callbacks=[tensorboard_callback])

Epoch 1/100
342/342 - 17s - loss: 22606898.0000 - val_loss: 61414736.0000 - 17s/epoch - 49ms/step
Epoch 2/100
342/342 - 14s - loss: 7990657.0000 - val_loss: 3699703.5000 - 14s/epoch - 40ms/step
Epoch 3/100
342/342 - 13s - loss: 4130298.7500 - val_loss: 136808.1094 - 13s/epoch - 38ms/step
Epoch 4/100
342/342 - 12s - loss: 2747299.2500 - val_loss: 35710.1680 - 12s/epoch - 35ms/step
Epoch 5/100
342/342 - 12s - loss: 2558378.2500 - val_loss: 3383.4780 - 12s/epoch - 36ms/step
Epoch 6/100
342/342 - 13s - loss: 1214455.8750 - val_loss: 111625.2891 - 13s/epoch - 37ms/step
Epoch 7/100
342/342 - 19s - loss: 337480.2500 - val_loss: 68686.6094 - 19s/epoch - 55ms/step
Epoch 8/100
342/342 - 15s - loss: 316366.7188 - val_loss: 2059.3557 - 15s/epoch - 44ms/step
Epoch 9/100
342/342 - 20s - loss: 293117.0312 - val_loss: 20961.5469 - 20s/epoch - 58ms/step
Epoch 10/100
342/342 - 17s - loss: 575945.1875 - val_loss: 503602.8438 - 17s/epoch - 50ms/step
Epoch 11/100
342/342 - 13s - loss: 290962.8750 - val_los

KeyboardInterrupt: 

In [39]:
model_path = 'C:/Code_Win/ML_DL_RL/LSTM_v2/model_save/3sat_LSTM_RTT_' + datetime.now().strftime("%Y%m%d-%H%M%S") + '_' + str(window_size) + '.h5'
model.save(model_path)

c:\Code_Win\ML_DL_RL\.venvML_DL_RL\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

predictions = model.predict([X_val, OBT_val])
rmse = sqrt(mean_squared_error(y_val, predictions))

predictions.shape, rmse

### underprogress

In [ ]:
np.set_printoptions(suppress=True, precision=4)
# y_pred = scaler.inverse_transform(y_pred)
temp = y_pred[23].reshape(2, 2)
print(temp)
temp = temp.round(0)
print(temp)

In [ ]:
def find_path(predecessor_matrix, source, destination):
    # Start at the destination node.
    node = destination
    path = [node]
    
    # Trace back through the predecessors until we reach the source.
    while node != source:
        # Fetch the predecessor of the current node for the path starting at the source.
        predecessor = int(predecessor_matrix[source][node])
        
        # If there's no valid predecessor, then no path exists.
        if predecessor == -9999:
            return None
        
        # Add the predecessor to the path and continue tracing back.
        path.insert(0, predecessor)
        node = predecessor
    
    return path

source = 0
destination = 1
print(find_path(temp, source, destination))


In [ ]:
predecessor_matrix = [
    [-9999.,    0.,    1.],
    [   1., -9999.,    1.],
    [   0.,    1., -9999.]
]

source = 2
destination = 0
print(find_path(predecessor_matrix, source, destination))

In [ ]:
def find_path_in_list_of_matrices(matrices, source, destination):
    for matrix_index, matrix in enumerate(matrices):
        path = find_path(matrix, source, destination)
        if path:
            return (matrix_index, path)
    return (None, None)

def find_path(matrix, source, destination):
    path = [destination]
    while destination != source:
        destination = int(matrix[destination])
        if destination == -9999:  # Unreachable node
            return None
        path.append(destination)
    path.reverse()
    return path

# Example usage:
matrices = [
    [
        [-9999.,    1.],
        [   0., -9999.]
    ],
    [
        [-9999.,    0.],
        [   1., -9999.]
    ]
]

source = 0
destination = 1

matrix_index, path = find_path_in_list_of_matrices(matrices, source, destination)
if matrix_index is not None:
    print(f"Path found in matrix {matrix_index}: {path}")
else:
    print("No path found in any matrix.")


In [ ]:
def extract_path(predecessors, source, destination):
    path = [destination]
    while destination != source:
        destination = int(predecessors[destination])
        if destination == -9999:  # Unreachable node
            print("No path exists!")
            return []
        path.append(destination)
    path.reverse()
    return path

# Example usage:
predecessors = y_pred[23]  # This is your predicted predecessors list.
source_node = 0  # example, you should replace with your source node.
dest_node = 4   # example, you should replace with your destination node.

path = extract_path(predecessors, source_node, dest_node)
print(path)

In [ ]:
y_pred_df = pd.DataFrame(y_pred[32])
y_pred_df.head(20)

In [ ]:
def find_path(model, timestamp, src_node, dst_node, max_future_steps=10):
    # Convert timestamp to cyclic features
    hours, minutes = timestamp.hour, timestamp.minute
    hour_sin = np.sin(hours * (2. * np.pi / 24))
    hour_cos = np.cos(hours * (2. * np.pi / 24))
    minute_sin = np.sin(minutes * (2. * np.pi / 60))
    minute_cos = np.cos(minutes * (2. * np.pi / 60))
    
    # Prepare the input vector (assuming the rest of the input remains constant)
    input_vector = [hour_sin, hour_cos, minute_sin, minute_cos, src_node, dst_node]
    input_vector = np.array(input_vector).reshape(1, -1)  # reshape to match the model's expected input shape
    
    # Scale the input_vector using the scaler used during training
    input_vector = scaler.transform(input_vector)
    
    # Predict using the model
    y_pred = model.predict(input_vector)
    
    # Extract path
    path = extract_path(y_pred[0], src_node, dst_node)
    if path:
        return path
    
    # If no path found, check for future instances
    for _ in range(max_future_steps):
        minutes += 1  # Increment time by one minute
        if minutes == 60:
            minutes = 0
            hours += 1
            if hours == 24:
                hours = 0
        # Update cyclic features
        hour_sin = np.sin(hours * (2. * np.pi / 24))
        hour_cos = np.cos(hours * (2. * np.pi / 24))
        minute_sin = np.sin(minutes * (2. * np.pi / 60))
        minute_cos = np.cos(minutes * (2. * np.pi / 60))
        
        input_vector = [hour_sin, hour_cos, minute_sin, minute_cos, src_node, dst_node]
        input_vector = np.array(input_vector).reshape(1, -1)
        input_vector = scaler.transform(input_vector)
        
        y_pred = model.predict(input_vector)
        path = extract_path(y_pred[0], src_node, dst_node)
        if path:
            return path
    
    print("No path found in the given or future instances.")
    return []

# Sample Usage
timestamp = datetime.time(14, 30)  # for example, 14:30
src_node = 0  # for example, source node is 0
dst_node = 4  # for example, destination node is 4
path = find_path(model, timestamp, src_node, dst_node)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_test, y_pred)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot of actual vs. predicted
plt.scatter(Y_test, y_pred)
plt.xlabel("Actual values")
plt.ylabel("Predicted values")
plt.title("Actual vs. Predicted")
plt.show()

In [ ]:
model.save('C:/Code_Win/ML_DL_RL/ML_ISL_LSTM/5sat_v1.h5')